___
## __Проект «Модель кредитного риск-менеджмента»__
___

### __2-я часть - Тестирование модели__ :
- файл: __`part_2_test_model.ipynb`__

---
### __Структура файла__

---    
* __Основная часть__
    * Загрузка модели из файла
---    
* __Проверка загруженной модели, тестирование на выборках из исходных данных__
    * Подготовка датасета с примерами для тестировани (на одном файле с исходными данными)
	* Тестирование модели (примерах из одного файла __`test_data/test_data.pq`__)
       * Результаты предсказаний записываются в файл __`test_data/prediction_results.csv`__
	* Полный цикл тестирования:
		* Чтение всех файлов с train-данными
		* Подготовка примеров для тестирования модели
        * Запись примеров в parquet-файл 
		* Предсказание.

In [30]:
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime

import dill
import dask.dataframe as dd

from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, mean_absolute_error, roc_curve, auc

In [31]:
%matplotlib inline
# matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

import warnings
from sklearn.exceptions import FitFailedWarning

# warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn")
warnings.filterwarnings("ignore", message="This Pipeline instance is not fitted yet")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 1000)

RANDOM_STATE = 42
TEST_SIZE = 0.3
MODEL_TUNING_TIME_LIMIT = 1800  # 30 минут в секундах
# IS_DEBUG = False
# -----------------------------------------------------------------------------------------------------------------
# Глобальные параметры (переменные)
# Пути к названия файлов с данными
PATH_TO_DATA_TRAIN = 'train_data/'
PATH_TO_DATA_TEST  = 'test_data/'
FILE_TRAIN_TARGET  = 'train_target.csv'
FINAL_DATASET      = 'final_dataset.pq'

s_bold = '\033[1m'

#### Чтение одного файла данных parquet

In [32]:
# Загрузка одного файла данных parquet
def read_dataset_one_file(a_filename, is_debug=False, is_memory_optimize=False):
    
    if not os.path.exists(a_filename):
        print(f'Файл {a_filename} не найден!')
        return None, None
        
    if is_debug:
        print(f'Чтение одного файла ... {a_filename}')
    
    time_start = time.time()
        
    df_temp = dd.read_parquet(a_filename)
    df_temp = df_temp.compute()

    # сброс индекса
    df_temp.reset_index(drop=True, inplace=True)
    
    if is_memory_optimize:    # оптимизация числовых типов данных
        memory_usage_before = get_memory_usage(df_temp, False)
        optimize_numeric(df_temp, is_debug)
        memory_usage_after = get_memory_usage(df_temp, False)
        if is_debug:
            print(f'\tОбъем памяти (ДО оптимизации типов данных и ПОСЛЕ):\t {round(memory_usage_before, 3)} Гб >> {round(memory_usage_after, 3)} Гб')

    time_read = round(time.time() - time_start, 2)
    if is_debug:
        print(f'Чтение завершено:\n\tОбщее время чтения и обработки = {time_read:.1f} сек;   Размер = {df_temp.shape}')
    
    return df_temp, time_read

#### Запись датасета в файл

In [33]:
# Запись датасета в файл
def save_dataset_to_file(a_df, a_path=PATH_TO_DATA_TEST, a_file=FINAL_DATASET):

    file_full_name = a_path + a_file
    print(f'Запись файла {file_full_name}...')
    
    time_start = time.time()
    a_df.to_parquet(file_full_name, index=False)
    
    time_save = time.time() - time_start
    print(f'Файл успешно записан; время записи = {time_save:.1f} сек\n')

#### Загрузка модели из файла

In [34]:
# Загрузка модели из файла
def load_model(filename='credit_risk_management_prediction_model.pkl'):
    s_folder = 'models/'
    filename = f'{s_folder}{filename}'
    
    with open(filename, 'rb') as file:
        pipe_model_loaded = dill.load(file)    
        return pipe_model_loaded

#### Получение объема памяти датафрейма

In [35]:
# Объем памяти датафрейма
def get_memory_usage(a_df, is_debug=True):
    memory_size = a_df.memory_usage(index=True).sum() / 10**9
    if is_debug:
        print(f"Объем памяти датафрейма: {round(memory_size, 3)} Гб")
    return memory_size

#### Оптимизация числовых типов данных датафрейма

In [36]:
# оптимизация чиловых типов данных
def optimize_numeric(a_df, is_debug=True):
    
    cols_int   = a_df.select_dtypes(include=['int']).columns
    cols_float = a_df.select_dtypes(include=['float']).columns
        # print(f'columns integer = {len(cols_int)}:\t', cols_int)
        # print(f'columns float = {len(cols_float)}:\t', cols_float)
    time_start = time.time()
   
    for col in cols_int:
        a_df[col] = pd.to_numeric( a_df[col], downcast='integer')

    for col in a_df.select_dtypes(include=['int', 'float']).columns:
        a_df[col] = pd.to_numeric( a_df[col], downcast='float')

    time_optim = time.time() - time_start
    
    if is_debug:
        print(f'\tОптимизация типов данных завершена; время оптимизации = {time_optim:.1f} сек')
        
    return a_df

#### Формирование датасета с примерами для тестирования модели (с балансом классов 0 и 1)

In [37]:
# Формирование датасета с примерами для тестирования модели (с балансом классов 0 и 1)
def prepare_test_dataset(df_source_data, df_target, n_samples_per_class=1000):
    """
    Формирование тестового датасета с балансом классов
    
    Parameters:
    -----------
        df_source_data : DataFrame
            Исходный датафрейм с признаками (может содержать несколько записей на один id)
        df_target : DataFrame
            Датафрейм с id и целевой переменной flag
        n_samples_per_class : int
            Количество примеров для каждого класса (по умолчанию 1000)
        
    Returns:
    --------
        df_test_features : DataFrame
            Датафрейм с признаками для тестирования (структура как в df_source_data) + столбец target
        df_test_answers : DataFrame
            Датафрейм с правильными ответами (id и flag)
    """
    
    # 1. Объединяем целевые метки с основным датафреймом
    df_merged = df_source_data.merge(df_target, on='id', how='inner')
    
    # 2. Разделяем по классам
    df_class_1 = df_merged[df_merged['flag'] == 1]
    df_class_0 = df_merged[df_merged['flag'] == 0]
    
    # 3. Получаем уникальные id для каждого класса
    unique_ids_1 = df_class_1['id'].unique()
    unique_ids_0 = df_class_0['id'].unique()
    
    print(f"Всего уникальных id с flag=1: {len(unique_ids_1)}")
    print(f"Всего уникальных id с flag=0: {len(unique_ids_0)}")
    
    # 4. Выбираем случайные id для каждого класса
    if len(unique_ids_1) < n_samples_per_class:
        print(f"Внимание: запрошено {n_samples_per_class} id с flag=1, но доступно только {len(unique_ids_1)}")
        selected_ids_1 = unique_ids_1
    else:
        selected_ids_1 = np.random.choice(unique_ids_1, n_samples_per_class, replace=False)
    
    if len(unique_ids_0) < n_samples_per_class:
        print(f"Внимание: запрошено {n_samples_per_class} id с flag=0, но доступно только {len(unique_ids_0)}")
        selected_ids_0 = unique_ids_0
    else:
        selected_ids_0 = np.random.choice(unique_ids_0, n_samples_per_class, replace=False)
    
    # 5. Отбираем все записи для выбранных id
    selected_ids = np.concatenate([selected_ids_1, selected_ids_0])
    df_test_features = df_merged[df_merged['id'].isin(selected_ids)].copy()
    
    # 6. Переименовываем 'flag' в 'target' и переставляем столбцы
    df_test_features = df_test_features.rename(columns={'flag': 'target'})
    
    # Создаем новый порядок столбцов: id, target, остальные столбцы
    columns_order = ['id', 'target'] + [col for col in df_test_features.columns if col not in ['id', 'target']]
    df_test_features = df_test_features[columns_order]
    
    # 7. Создаем датафрейм с правильными ответами (переименовываем для consistency)
    df_test_answers = df_target[df_target['id'].isin(selected_ids)].copy()
    df_test_answers = df_test_answers.rename(columns={'flag': 'target'})
    
    # 8. Статистика
    print(f"\nРезультат подготовки тестового датасета:")
    print(f"\tОтобрано уникальных id с target=1: {len(selected_ids_1)}")
    print(f"\tОтобрано уникальных id с target=0: {len(selected_ids_0)}")
    print(f"\n\tВсего записей в тестовом датафрейме: {len(df_test_features)}")
    print(f"\tВсего уникальных id в тестовых ответах: {len(df_test_answers)}")
    print(f"\nРаспределение target в тестовых признаках:")
    print(df_test_features['target'].value_counts().sort_index())
    
    return df_test_features, df_test_answers

#### Тестирование модели на готовых примерах

In [38]:
# Тестирование модели на готовых примерах
def test_model_predictions(model, 
                           data_samples, 
                           target_col='target', 
                           is_save_results_to_file=True):
    """
    Функция для тестирования модели на N * 2 случайных примерах с расчетом метрик
    
    Параметры:
        - model      : обученная модель
        - data       : DataFrame с данными
        - target_col : название целевой колонки
    
    Возвращает:
        - results    : DataFrame с предсказаниями
        - metrics    : словарь с метриками
    """
    #--------------------------------------------------------------------------------------------------
    if data_samples.empty:
        print('Нет примеров для тестирования!')
        return None, None
    
    # Деление данных x и y
    x_test = data_samples.drop(columns=[target_col])
    y_true = data_samples[target_col]

    print('x_test.shape, y_true.shape:', x_test.shape, y_true.shape)
    
    #--------------------------------------------------------------------------------------------------
    # Предсказание
    y_pred = model.predict(x_test)
    
    #  id для группировки
    if 'id' in data_samples.columns:
        
        # Группировка y_true по id (берем первое значение для каждого id)
        y_true_grouped = data_samples.groupby('id')[target_col].first()
        
        # Проверка соответствия размеров
        print(f'После группировки: y_true_grouped={y_true_grouped.shape}, y_pred={len(y_pred)}')
        
        # Если все еще не совпадают, выравнивание по минимальному размеру
        if len(y_true_grouped) != len(y_pred):
            min_len = min(len(y_true_grouped), len(y_pred))
            y_true_grouped = y_true_grouped.iloc[:min_len]
            y_pred = y_pred[:min_len]
            print(f'После выравнивания: y_true_grouped={len(y_true_grouped)}, y_pred={len(y_pred)}')
    else:
        # Если нет колонки 'id', просто выравниваем по минимальному размеру
        min_len = min(len(y_true), len(y_pred))
        y_true_grouped = y_true.iloc[:min_len]
        y_pred = y_pred[:min_len]
        print(f'Выравнивание без ID: y_true_grouped={len(y_true_grouped)}, y_pred={len(y_pred)}')

    #--------------------------------------------------------------------------------------------------
    # Проверка возможности расчета roc_auc
    roc_auc = None
    if hasattr(model, 'predict_proba'):
        try:
            y_proba = model.predict_proba(x_test)[:, 1]
            
            # группирока y_proba, если нужно
            if 'id' in data_samples.columns and len(y_proba) != len(y_true_grouped):
                min_len = min(len(y_true_grouped), len(y_proba))    # Если y_proba имеет другой размер, тоже группировка
                y_proba = y_proba[:min_len]
                y_true_for_proba = y_true_grouped.iloc[:min_len]
            else:
                y_true_for_proba = y_true_grouped
                
            roc_auc = roc_auc_score(y_true_for_proba, y_proba)
        except Exception as e:
            print(f'Ошибка расчета ROC-AUC: {e}')
    
    # Расчет метрики с группированными данными
    # accuracy = accuracy_score(y_true_grouped, y_pred)
    
    #--------------------------------------------------------------------------------------------------
    # DataFrame с результатами тестирования
    results = pd.DataFrame({
        'y_valid':    y_true_grouped,
        'y_predict':  y_pred,
        'is_correct': (y_true_grouped == y_pred).astype(int)
    })
    
    # Добавление столбца id
    if 'id' in data_samples.columns:
        results['id'] = y_true_grouped.index.astype(int)
    
    # Добавление столбца с вероятностью
    if hasattr(model, 'predict_proba') and 'y_proba' in locals():
        results['prediction_prob'] = y_proba[:len(results)]  # Обрезаю до нужного размера

    results.reset_index(drop=True, inplace=True)
    
    # Метрики и количество примеров
    metrics = {
        # 'accuracy': accuracy,
        'roc_auc': roc_auc,
        'n_samples': len(results)
    }

    #--------------------------------------------------------------------------------------------------
    # Запись результатов предсказаний в файл
    if is_save_results_to_file:
        results.to_csv(PATH_TO_DATA_TEST + 'prediction_results.csv', index=False)
        print(f'Результаты тестирования записаны в файл: "{PATH_TO_DATA_TEST + 'prediction_results.csv'}"')
    
    #--------------------------------------------------------------------------------------------------
    # Вывод результатов
    print('-' * 100)
    print(f'\nРезультаты тестирования на {len(results)} примерах:')
    # print(f'\tAccuracy: {accuracy:.6f}')
    
    if roc_auc is not None:
        print(f'\troc_auc: {roc_auc:.6f}\n')
    print('-' * 100)
    
    return results, metrics

#### Подготовка одного файла с сырыми данными для тестирования модели - выжимка из тренировочных файлов

In [39]:
# Подготовка одного файла с сырыми данными для тестирования модели - выжимка из тренировочных файлов
# + тестирование модели на каждом из файлов
def prepare_file_for_test_predict(n_files = 2, n_samples = 1000, pipe_model = None):
    """
    # Подготовка файла с данными для тестирования
        # перебор n_files файлов c train-данными
        # подготовка по каждому файлу датсета с примерами (на каждый класс - по n_samples примеров)
        # если нужно, то тестирование предсказаний модели для каждого датасета (pipe_model is not None)
        # запись всех тестовых датасетов в один parquet-файл 'test_data.pq'
    """    
    metrics_all_tests = []
    test_data_all = []
    df_test_data_all = None
    df_metrics_all_tests = None

    dataset_paths = sorted(
        [os.path.join( PATH_TO_DATA_TRAIN, filename) for filename in os.listdir( PATH_TO_DATA_TRAIN)
         if filename.startswith('train_data_') and filename.endswith('.pq')],
        key=lambda x: int(x.split('_')[-1].split('.')[0])
    )
    
    if len(dataset_paths) == 0:
        print(f'В папке {PATH_TO_DATA_TRAIN} не найдено файлов вида "train_data_*.pq"')
        return None

    file_name = PATH_TO_DATA_TRAIN + FILE_TRAIN_TARGET
    if not os.path.exists(file_name):
        print(f'Файл {file_name} ней найден!')
        return None

    n_files = max(0, n_files)
    chunks = dataset_paths[:n_files]

    for i, file_name in enumerate(chunks):
    # for i in range(n_files):
        print(f'file {i}:')
        #------------------------------------------------------------------------------------------------------------
        # чтение файла c train-данными
        # df_data_raw, time_read = read_dataset_one_file( PATH_TO_DATA_TRAIN + f'train_data_{i}.pq', 
        df_data_raw, time_read = read_dataset_one_file( file_name, 
                                                       is_debug=True, 
                                                       is_memory_optimize=True)
        df_target = pd.read_csv( PATH_TO_DATA_TRAIN + FILE_TRAIN_TARGET )   

        #------------------------------------------------------------------------------------------------------------
        # подготовка датсета с примерами (на каждый класс - по n_samples_per_class примеров)
        df_test_data, df_test_answers = prepare_test_dataset(df_data_raw, df_target, n_samples_per_class=5000)    
        test_data_all.append(df_test_data)  # ← СОБИРАЕМ данные

        #------------------------------------------------------------------------------------------------------------
        # тестирование модели
        if (pipe_model is not None):
            results, metrics = test_model_predictions(model=pipe_model, 
                                                      data_samples=df_test_data, target_col='target', 
                                                      is_save_results_to_file=False)
            # results, metrics = test_model_predictions(model=pipe_model_loaded['model'], 
                                                        # data_samples=df_test_data, target_col='target', 
                                                        # is_save_results_to_file=False)
            metrics_all_tests.append(metrics)

        #
        if df_test_data_all is None:
            df_test_data_all = df_test_data
        else:
            df_test_data_all = pd.concat( [df_test_data_all, df_test_data], ignore_index=True)
            # df_result = pd.concat( [df_result, df_temp_processed], ignore_index=True)
            
    # все данные в один датафрейм
    # df_test_data_all     = pd.concat(test_data_all, ignore_index=True)
    df_metrics_all_tests = pd.DataFrame(metrics_all_tests)
    
    # запись датасета со всеми примерами для тестирования в отдельный файл
    save_dataset_to_file( a_df=df_test_data_all, 
                          a_path=PATH_TO_DATA_TEST, 
                          a_file='test_data.pq')
    
    # результаты
    print('=' * 100)
    print(f"Объединенный датафрейм:\n\tРазмеры = {df_test_data_all.shape};\tЗаписан в файл: {PATH_TO_DATA_TEST + 'test_data.pq'}")
    
    if (pipe_model is not None):
        print(f"Метрики по файлам:")
        print(df_metrics_all_tests)
    
        print(f'По всем {len(df_metrics_all_tests)} файлам-примерам:\n\troc_auc mean = {df_metrics_all_tests.roc_auc.mean():.6f}')
        
    return df_metrics_all_tests #, df_test_data_all
    # return df_metrics_all_tests, df_test_data_all

## Основная часть

### Проверка. Загрузка модели из файла

In [40]:
# Проверка - Загрузка модели из файла
pipe_model_loaded = load_model('credit_risk_management_prediction_model.pkl')
pipe_model_loaded

{'model': Pipeline(steps=[('preprocessor',
                  Pipeline(steps=[('data_preprocessing',
                                   FunctionTransformer(func=<function data_preprocessing at 0x000001E4AEA33C40>)),
                                  ('selector_features',
                                   FeatureSelectorTransformer(all_top_features=['ratio_overdue_loans3060',
                                                                                'ratio_overdue_loans90',
                                                                                'ratio_overdue_loans6090',
                                                                                'ratio_loans_overdue_total',
                                                                                'ratio_overdue_loans530',
                                                                                'count_ov...
                                                              max_cat_threshold=None,
                

In [41]:
pipe_model_loaded['model']

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,steps,"[('data_preprocessing', ...), ('selector_features', ...), ...]"
,transform_input,None
,memory,None
,verbose,True
,func,<function dat...001E4AEA33C40>
,inverse_func,None
,validate,False


In [42]:
pipe_model_loaded['model'].named_steps['classifier']

,estimators,"[('LGBMCLassifier objective_fast', ...), ('XGBClassifier objective_xgb_fast', ...), ...]"
,voting,'soft'
,weights,"[1, 1, ...]"
,n_jobs,-1
,flatten_transform,True
,verbose,False
,boosting_type,'gbdt'
,num_leaves,59
,max_depth,7
,learning_rate,0.02424703456435817
,n_estimators,1650


#### Проверка атрибутов пайплайна и модели

In [46]:
# Проверка, что модель загружена корректно
print(f'Тип модели: {type(pipe_model_loaded['model'])}')
print(f'Тип классификатора: {type(pipe_model_loaded['model'].named_steps['classifier']).__name__}')

attr = [attr for attr in dir(pipe_model_loaded['model']) if 'fit' in attr or 'class' in attr]
# print(f"\nАтрибуты обученности: {attr}")
# Проверка атрибутов, указывающих на обучение
if False:
    print("Проверка атрибутов:")
    for item in attr:
        print(f"\thasattr(model, '{item:<22}'): {hasattr(pipe_model_loaded['model'], item)}")

Тип модели: <class 'sklearn.pipeline.Pipeline'>
Тип классификатора: VotingClassifier


#### Получение параметров загруженной модели

In [47]:
model_params = pipe_model_loaded['model'].named_steps['classifier'].get_params()
print('Параметры - список:')
print(f'model_params = {model_params}\n')

# Получение параметров загруженной модели
model_params = pipe_model_loaded['model'].named_steps['classifier'].get_params()

df_params = pd.DataFrame({
    'Параметр': model_params.keys(),
    'Значение': model_params.values()
})

print('Параметры - таблица:')
print(df_params)

Параметры - список:
model_params = {'estimators': [('LGBMCLassifier objective_fast', LGBMClassifier(colsample_bytree=0.7248044994766626, is_unbalance=False,
               learning_rate=0.02424703456435817, max_depth=7,
               min_child_samples=70, n_estimators=1650, n_jobs=-1,
               num_leaves=59, random_state=42, reg_alpha=0.738924554711979,
               reg_lambda=0.7647132769869521, subsample=0.729283573622419,
               verbose=-1)), ('XGBClassifier objective_xgb_fast', XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8346382656042973, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.033019071561003456,
              max_bin=None, max_cat_thr

#### Тестирование модели на примерах из готового файла
- `test_data.pq` (выжимка из файлов с train-данными)

In [48]:
# Загрузка файла с данными для тестирования модели 
file_name = PATH_TO_DATA_TEST + 'test_data.pq'

df_test_data, time_read = read_dataset_one_file( 
                                                    a_filename=file_name, 
                                                    is_debug=True, 
                                                    is_memory_optimize=True
)
if df_test_data is not None:
    
    # file_name = PATH_TO_DATA_TRAIN + FILE_TRAIN_TARGET
    # if os.path.exists(file_name):
        
    #     print(f'Чтение файла {file_name}...')
    #     df_target = pd.read_csv( file_name )
    #     optimize_numeric(df_target, is_debug=False)
        
    # Тестирование модели на примерах из одного файла
    results, metrics = test_model_predictions(
                                                model        = pipe_model_loaded['model'], 
                                                data_samples = df_test_data, 
                                                target_col   = 'target',
                                                is_save_results_to_file = True
    )
    
    print(metrics)
    print('\nПример результатов предсказаний:')
    print(results[results['y_valid'] == 1].sort_values('is_correct', ascending=False).head())
        
    # else:
    #     print(f'Файл {file_name} не найден!')

Чтение одного файла ... test_data/test_data.pq
	Оптимизация типов данных завершена; время оптимизации = 0.1 сек
	Объем памяти (ДО оптимизации типов данных и ПОСЛЕ):	 0.074 Гб >> 0.066 Гб
Чтение завершено:
	Общее время чтения и обработки = 0.4 сек;   Размер = (1021346, 62)
x_test.shape, y_true.shape: (1021346, 61) (1021346,)
----------------------------------------------------------------------------------------------------
Выполняется data_preprocessing ...
	Группировка данных по столбцу "rn"...
	Создание новых признаков ...
	Выполняется ohe_source_columns ...
		ohe - 1 - source df_ids shape = (1021346, 1)
		ohe - 2 - fit
		ohe - 3 - transform; df_ids shape = (1021346, 1)
		ohe - 4 - df_ids: group & sum; df_ids shape = (120000, 408)
		ohe - 5 - df_total - merge; df_total.shape = (120000, 428)
	ohe_source_columns завершен:	   Время выполнения = 7.3s;	   shape = (120000, 428)
----------------------------------------------------------------------------------------------------
Выполняется 

In [27]:
results.head(3)

,y_valid,y_predict,is_correct,id,prediction_prob
0,1,0,0,20,0.099478
1,1,0,0,118,0.075861
2,1,0,0,120,0.072880


#### Полный цикл тестирования
- Чтение всех файлов с train-данными
- Подготовка примеров для тестирования модели
- Запись примеров в файл `test_data.pq`
- Предсказание для каждого файла в отдельности

In [24]:
%%time
# Подготовка файла с данными для тестирования
    # перебор n_files файлов c train-данными
    # подготовка датсетов с примерами (на каждый класс - по n_samples примеров)
    # тестирование предсказания модели по каждому датасету (если это нужно, то pipe_model is not None)
    # запись тестовых датасетов в parquet-файл 'test_data.pq'

#--------------------------------------------------------------------------------------------------------------
# Запуск генерации примеров сразу с предсказанием по каждому файлу в отдельности
if True:
    df_metrics_all_tests = prepare_file_for_test_predict(
                                                            n_files = 12, 
                                                            n_samples = 5000, 
                                                            pipe_model = pipe_model_loaded['model']
    )
#--------------------------------------------------------------------------------------------------------------
# Запуск генерации примеров (без предсказаний, формируется только файл 'test_data.pq')
if False:
    _ = prepare_file_for_test_predict(
                                        n_files = 12, 
                                        n_samples = 5000, 
                                        pipe_model = None
    )

file 0:
Чтение одного файла ... train_data/train_data_0.pq
	Оптимизация типов данных завершена; время оптимизации = 1.5 сек
	Объем памяти (ДО оптимизации типов данных и ПОСЛЕ):	 0.964 Гб >> 0.126 Гб
Чтение завершено:
	Общее время чтения и обработки = 2.6 сек;   Размер = (1974724, 61)
Всего уникальных id с flag=1: 7734
Всего уникальных id с flag=0: 242266

Результат подготовки тестового датасета:
	Отобрано уникальных id с target=1: 5000
	Отобрано уникальных id с target=0: 5000

	Всего записей в тестовом датафрейме: 76692
	Всего уникальных id в тестовых ответах: 10000

Распределение target в тестовых признаках:
target
0    39342
1    37350
Name: count, dtype: int64
x_test.shape, y_true.shape: (76692, 61) (76692,)
----------------------------------------------------------------------------------------------------
Выполняется data_preprocessing ...
	Группировка данных по столбцу "rn"...
	Создание новых признаков ...
	Выполняется ohe_source_columns ...
		ohe - 1 - source df_ids shape = (766

In [25]:
if (df_metrics_all_tests is not None):
    print(f'По всем {len(df_metrics_all_tests)} файлам-примерам:\n\troc_auc mean = {df_metrics_all_tests.roc_auc.mean():.6f}')

По всем 12 файлам-примерам:
	roc_auc mean = 0.779953
